# Spark101 - Exercises

1. Create a spark data frame that contains your favorite programming languages.

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 13:14:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/08 13:14:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/08 13:14:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
import pandas as pd
import numpy as np

- The name of the column should be language

In [3]:
data = ['python', 'java', 'c++', 'html', 'r', 'nlp']
  
pandas_df = pd.DataFrame(data, columns=['language'])
  
pandas_df

,language
0,python
1,java
2,c++
3,html
4,r
5,nlp


In [4]:
spark_df = spark.createDataFrame(pandas_df)
spark_df

DataFrame[language: string]

- View the schema of the dataframe

In [5]:
spark_df.printSchema()

root
 |-- language: string (nullable = true)



- Output the shape of the dataframe

In [6]:
print((spark_df.count(), len(spark_df.columns)))

(6, 1)


- Show the first 5 records in the dataframe

In [7]:
spark_df.show(5)

+--------+
|language|
+--------+
|  python|
|    java|
|     c++|
|    html|
|       r|
+--------+
only showing top 5 rows



2. Load the mpg dataset as a spark dataframe.

In [8]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



A. Create 1 column of output that contains a message like the one below:

The 1999 audi a4 has a 4 cylinder engine.  

For each vehicle.  

In [9]:
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, expr

In [10]:
mpg_message = mpg.select(concat(lit('The '), mpg.year, lit(' '), mpg.manufacturer, lit(' '), mpg.model, lit(' has a '), mpg.cyl, lit(' cylinder engine.'))).show(5, truncate=False)

+------------------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a , cyl,  cylinder engine.)|
+------------------------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                                     |
|The 1999 audi a4 has a 4 cylinder engine.                                     |
|The 2008 audi a4 has a 4 cylinder engine.                                     |
|The 2008 audi a4 has a 4 cylinder engine.                                     |
|The 1999 audi a4 has a 6 cylinder engine.                                     |
+------------------------------------------------------------------------------+
only showing top 5 rows



B. Transform the trans column so that it only contains either manual or auto.

In [11]:
from pyspark.sql.functions import when

In [12]:
mpg.select(
    mpg.trans,
    when(mpg.trans.startswith('a') == True, "auto")
    .otherwise("manual")
    .alias("trans"),
).show(12)

+----------+------+
|     trans| trans|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
|manual(m5)|manual|
|  auto(av)|  auto|
|manual(m5)|manual|
|  auto(l5)|  auto|
|manual(m6)|manual|
|  auto(s6)|  auto|
|  auto(l5)|  auto|
+----------+------+
only showing top 12 rows



3. Load the tips dataset as a spark dataframe.

In [47]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



A. What percentage of observations are smokers?

In [48]:
total_smoker = tips.select(tips.smoker).count()

In [49]:
smokes_yes = tips.filter(tips.smoker == 'Yes').count()

In [50]:
(smokes_yes / total_smoker) * 100

38.114754098360656

B. Create a column that contains the tip percentage

In [51]:

tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [57]:
tips.withColumn("tip_percentage", (tips.tip / tips.total_bill) * 100).show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|    tip_percentage|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883

C. Calculate the average tip percentage for each combination of sex and smoker.

In [55]:
new_df.show()

AttributeError: 'NoneType' object has no attribute 'show'

In [40]:
tips.filter(tips.smoker == 'No').where(tips["sex"] == "Female").show()

AttributeError: 'NoneType' object has no attribute 'filter'

4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.

- Which month has the most rain, on average?

- Which year was the windiest?

- What is the most frequent type of weather in January?

- What is the average high and low temperature on sunny days in July in 2013 and 2014?

- What percentage of days were rainy in q3 of 2015?

- For each year, find what percentage of days it rained (had non-zero precipitation).